In [1]:
import torch
import os
import cv2
import json
import numpy as np

import albumentations as A
import torch.nn.functional as F

import pydensecrf.densecrf as dcrf
import matplotlib.pyplot as plt

from pydensecrf.utils import unary_from_labels
from matplotlib.patches import Patch

from skimage.color import gray2rgb
from hrnet import *

In [2]:
def decode_rle_to_mask(rle, height, width):
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(height * width, dtype=np.uint8)
    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(height, width)

def encode_mask_to_rle(mask):
    """
    mask: numpy array binary mask
    1 - mask
    0 - background
    Returns encoded run length
    """
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)

PALETTE = [
    (220, 20, 60), (119, 11, 32), (0, 0, 142), (0, 0, 230), (106, 0, 228),
    (0, 60, 100), (0, 80, 100), (0, 0, 70), (0, 0, 192), (250, 170, 30),
    (100, 170, 30), (220, 220, 0), (175, 116, 175), (250, 0, 30), (165, 42, 42),
    (255, 77, 255), (0, 226, 252), (182, 182, 255), (0, 82, 0), (120, 166, 157),
    (110, 76, 0), (174, 57, 255), (199, 100, 0), (72, 0, 118), (255, 179, 240),
    (0, 125, 92), (209, 0, 151), (188, 208, 182), (0, 220, 176),
]
CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]
def label2rgb(label):
    image_size = label.shape[1:] + (3, )
    image = np.zeros(image_size, dtype=np.uint8)
    
    for i, class_label in enumerate(label):
        image[class_label == 1] = PALETTE[i]
        
    return image

def get_inner_files(path, extension):
    files = {
        os.path.relpath(os.path.join(root, fname), start=path)
        for root, _, files in os.walk(path)
        for fname in files
        if os.path.splitext(fname)[1].lower() == f".{extension}"
    }
    return sorted(files)

In [3]:
# from https://www.kaggle.com/code/meaninglesslives/apply-crf/notebook
def crf(W, H, mask_img):
    
    # Converting annotated image to RGB if it is Gray scale
    if(len(mask_img.shape)<3):
        mask_img = gray2rgb(mask_img)
#     #Converting the annotations RGB color to single 32 bit integer
    annotated_label = mask_img[:,:,0] + (mask_img[:,:,1]<<8) + (mask_img[:,:,2]<<16)
    
#     # Convert the 32bit integer color to 0,1, 2, ... labels.
    colors, labels = np.unique(annotated_label, return_inverse=True)

    n_labels = 2
    
    #Setting up the CRF model
    d = dcrf.DenseCRF2D(H, W, n_labels)

    # get unary potentials (neg log probability)
    U = unary_from_labels(labels, n_labels, gt_prob=0.7, zero_unsure=False)
    d.setUnaryEnergy(U)

    # This adds the color-independent term, features are the locations only.
    d.addPairwiseGaussian(sxy=(4, 3), compat=3, kernel=dcrf.FULL_KERNEL,
                      normalization=dcrf.NORMALIZE_SYMMETRIC)
        
    #Run Inference for 5 steps 
    Q = d.inference(5)

    # Find out the most probable class for each pixel.
    MAP = np.argmax(Q, axis=0)

    return MAP.reshape((W, H))

In [4]:
torch.cuda.empty_cache()

In [5]:
images_path = '/opt/ml/level2_cv_semanticsegmentation-cv-01/data/train/DCM'
labels_path = '/opt/ml/level2_cv_semanticsegmentation-cv-01/data/train/outputs_json'

filenames = np.array(get_inner_files(images_path, "png"))
labelnames = np.array(get_inner_files(labels_path, "json"))
class2ind = {v: i for i, v in enumerate(CLASSES)}
i = 0

In [6]:
def get(i, tf):
    image_name = filenames[i]
    image_path = os.path.join(images_path, image_name)

    original_image = cv2.imread(image_path)
    image = original_image / 255.0

    label_name = labelnames[i]
    label_path = os.path.join(labels_path, label_name)
    label_shape = tuple(image.shape[:2]) + (len(CLASSES),)
    label = np.zeros(label_shape, dtype=np.uint8)

    # read label file
    with open(label_path, "r") as f:
        annotations = json.load(f)
    annotations = annotations["annotations"]

    # iterate each class
    for ann in annotations:
        c = ann["label"]
        class_ind = class2ind[c]
        points = np.array(ann["points"])

        # polygon to mask
        class_label = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.fillPoly(class_label, [points], 1)
        label[..., class_ind] = class_label

    inputs = ({"image": image})
    result = tf(**inputs)

    image = result["image"]
    image = image.transpose(2, 0, 1)
    label = label.transpose(2, 0, 1)
    image = torch.from_numpy(image).cuda()
    return image, label

In [7]:
model1 = torch.load("/opt/ml/level2_cv_semanticsegmentation-cv-01/pretrain/hrnet_48w_ocr_1024_augSet_Ensemble_best0.pth")
model2 = torch.load("/opt/ml/level2_cv_semanticsegmentation-cv-01/pretrain/hrnet_48w_ocr_1024_augSet_Ensemble_best1.pth")
model3 = torch.load("/opt/ml/level2_cv_semanticsegmentation-cv-01/pretrain/hrnet_48w_ocr_1024_augSet_Ensemble_best2.pth")
model4 = torch.load("/opt/ml/level2_cv_semanticsegmentation-cv-01/pretrain/hrnet_48w_ocr_1024_augSet_Ensemble_best3.pth")
model5 = torch.load("/opt/ml/level2_cv_semanticsegmentation-cv-01/pretrain/hrnet_48w_ocr_1024_augSet_Ensemble_best4.pth")
model1 = model1.cuda()
model2 = model2.cuda()
model3 = model3.cuda()
model4 = model4.cuda()
model5 = model5.cuda()
model1.eval()
model2.eval()
model3.eval()
model4.eval()
model5.eval()

HighResolutionNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): SyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): 

In [8]:
tf = A.Compose(
        [
            A.Resize(1024, 1024),
            A.Normalize(
                mean=(0.121, 0.121, 0.121),
                std=(0.1641, 0.1641, 0.1641),
                max_pixel_value=1,
            ),
        ]
    )

In [ ]:
image, label = get(i, tf)
image = image.unsqueeze(0)

outputs = model5(image)
outputs = F.interpolate(outputs, size=(2048, 2048), mode="bilinear")
outputs = torch.sigmoid(outputs)
outputs = (outputs > 0.5).detach().cpu().numpy()

for i in range(outputs.shape[0]):
    for j in range(outputs.shape[1]):
        outputs[i][j] = crf(2048, 2048, outputs[i][j])

outputs = outputs.reshape(29, 2048, 2048)
different_elements = np.sum(np.not_equal(outputs, label))
total_pixel = 29*2048*2048
print(f"서로 다른 값의 개수: {different_elements}/{total_pixel}({different_elements/total_pixel})")

fig, ax = plt.subplots(1, 2, figsize=(24, 12))
ax[0].imshow(label2rgb(label))
ax[1].imshow(label2rgb(outputs))

legend_elements = [Patch(facecolor=[c / 255 for c in color], label=CLASSES[i]) for i, color in enumerate(PALETTE)]
plt.legend(handles=legend_elements)
plt.show()

In [11]:
origin, crf = [], []
for k in range(10):
    image, label = get(k, tf)
    image = image.unsqueeze(0)

    outputs = model5(image)
    outputs = F.interpolate(outputs, size=(2048, 2048), mode="bilinear")
    outputs = torch.sigmoid(outputs)
    outputs = (outputs > 0.5).detach().cpu().numpy()
    
    copy_outputs = outputs.reshape(29, 2048, 2048)
    different_elements = np.sum(np.not_equal(copy_outputs, label))
    origin.append(different_elements)
    print(outputs.shape)
    for i in range(outputs.shape[0]):
        for j in range(outputs.shape[1]):
            outputs[i][j] = crf(2048, 2048, outputs[i][j])

    outputs = outputs.reshape(29, 2048, 2048)
    different_elements = np.sum(np.not_equal(outputs, label))
    crf.append(different_elements)
total_pixel = 29*2048*2048
print(np.mean(origin)/total_pixel)
print(np.mean(crf)/total_pixel)

(1, 29, 2048, 2048)


TypeError: 'list' object is not callable